In [1]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
def process_data(file):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    return np.array(coord)

In [160]:
def read_directory(directory, sample_rate):
    point_array_all = []
    land_mark_array_all=[]

    for filename in sorted(os.listdir(directory)):

        if filename.endswith(".pts"): 
            path=os.path.join(directory, filename)
            #print("Processing:",path)
            points = open(path)
            point_array = process_data(points)

            point_array_sampled = []
            
            #sampling
            for i in range(0,len(point_array),sample_rate):
                point_array_sampled.append(point_array[i])

            point_array_sampled = np.array(point_array_sampled)

            point_array_all.append(point_array_sampled)

            continue
        elif filename.endswith(".lan"):
            path = os.path.join(directory, filename)
            #print("Processing:",path)
            landmarks = open(path)
            landmark_array = process_data(landmarks)

            land_mark_array_all.append(landmark_array)

            continue
            
    return point_array_all,land_mark_array_all

In [108]:
def calculate_transformation_matrix(original, aligned):
    transform_matrices=[]
    for i in range(0, len(original)):
        transform_matrix = np.zeros(shape=(3,4))
        
        transform_matrix[0][-1] = aligned[i][0][0] - original[i][0][0]
        transform_matrix[0][0] = 1
        
        transform_matrix[1][-1] = aligned[i][0][1] - original[i][0][1]
        transform_matrix[1][1] = 1
        
        transform_matrix[2][-1] = aligned[i][0][2] - original[i][0][2]
        transform_matrix[2][2] = 1
        
        transform_matrices.append(transform_matrix)
    
    return transform_matrices

In [139]:
def calculate_aligned_landmarks(landmarks, transform_matrices):
    updated_landmarks_all = []
    
    for i in range(0, len(landmarks)):
        updated_landmarks=[]
        
        if(len(landmarks[i])!=0):
            landmarks_1 = np.c_[landmarks[i],np.ones(len(landmarks[i]))]
                
            for j in range(0, len(landmarks[i])):
                updated_landmark=[]
                
                for row in transform_matrices[i]:
                    updated_landmark.append(np.dot(landmarks_1[j],row))
           
                
                updated_landmarks.append(updated_landmark)
            updated_landmarks_all.append(updated_landmarks)
        
    return np.array(updated_landmarks_all, dtype=object)

In [351]:
def length_width_calculator(foot_data):
    
    threshold = 0.5
    
    lengths = []
    widths = []
    
    for foot in foot_data:
        point_x = foot[:,0]
        point_z = foot[:,2]

        length = 0
        for point in range(int(min(point_x)),int(max(point_x))):
            indices_filtered = np.where((point_x>=point-threshold)&(point_x<=point+threshold))
            point_z_filtered = point_z[indices_filtered]
            
            if(len(point_z_filtered)>0):
                length_point = max(point_z_filtered) - min(point_z_filtered)
            if length_point>length:
                length = length_point
                
        lengths.append(length)
                
        width = 0
        for point in range(int(min(point_z)),int(max(point_z))):
            indices_filtered = np.where((point_z>=point-threshold)&(point_z<=point+threshold))
            point_x_filtered = point_x[indices_filtered]
            
            if(len(point_x_filtered)>0):
                width_point = max(point_x_filtered) - min(point_x_filtered)
            if width_point>width:
                width = width_point
        
        widths.append(width)
    
    return lengths,widths

In [352]:
directory_aligned = "Datasets/Original Foot Data/Bounding Box Aligned/"
directory_processed = "Datasets/Original Foot Data/Processed/"

In [353]:
sample_rate = 1 #read by skipping given number of lines

In [354]:
aligned_points_all = read_directory(directory_aligned, sample_rate)[0]
original_points_all, original_landmarks_all = read_directory(directory_processed, sample_rate)

In [355]:
trans_mats = calculate_transformation_matrix(original_points_all, aligned_points_all)

In [356]:
aligned_landmarks_all = calculate_aligned_landmarks(original_landmarks_all,trans_mats)

In [357]:
point_x = aligned_points_all[0][:,0]
point_z = aligned_points_all[0][:,2]

In [358]:
threshold = 0.5
point = (max(point_x) + min(point_x))/2

In [359]:
#np.where(np.logical_and(a>=6, a<=10))

In [360]:
lengths,widths = length_width_calculator(aligned_points_all)

In [386]:
fig2 = plt.figure()
ax = fig2.add_subplot()
ax.scatter(range(len(lengths)),lengths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [388]:
fig2 = plt.figure()
ax = fig2.add_subplot()
ax.scatter(range(len(lengths)),widths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [389]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(range(len(lengths)),lengths,widths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …